In [ ]:
import numpy as np
import networkx as nx
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pydot
import meshplot as mp
from networkx.drawing.nx_pydot import graphviz_layout
from itertools import combinations


In [ ]:
branches = []
borders = {}
with open( '../results/faust_6890/faust_6890.txt' ) as file:
    for idx,line in enumerate(file):
        line =line.replace('\n', '')
        contents = line.split(" ")


        if tuple(contents[1:4]) in borders:
            borders[ tuple(contents[1:4]) ].add(idx)
        else:
            borders[ tuple(contents[1:4]) ] = set([idx])

        if tuple(contents[-3:]) in borders:
            borders[ tuple(contents[-3:]) ].add(idx)
        else:
            borders[ tuple(contents[-3:]) ] = set([idx])

        branches.append( np.reshape( np.array( contents[1:]).astype(np.float32), ((len(contents) - 1) // 3, 3) ) )

len(branches)

In [ ]:
cmap = cm.get_cmap('flag')
p = mp.plot( np.array([0,0,0]), shading={'point_size':0.00001}, return_plot=True)
for i, branch in enumerate(branches):
    c = '#'
    for x in (np.array(cmap( (i)/len(branches)) )[0:3] * 255).astype(np.uint32):
        h = hex(x)[2:]
        c += h if len(h) == 2 else '0' + h

    p.add_points( np.array(branch), shading={'point_size': 0.1, 'point_color':c })
    p.add_points( np.array([ branch[0], branch[-1]]) + np.random.rand(2,3) * 0.01, shading={'point_size': 0.2, 'point_color':c } )



In [ ]:
grafo = nx.Graph()
grafo.add_nodes_from( np.arange(0,len(branches)) )
grafo.add_edges_from( [ t for s in borders.values() for t in combinations(s, 2) ] )


In [ ]:
nx.draw(grafo, with_labels=True)

In [ ]:
nx.number_connected_components(grafo)

In [ ]:
print( len(grafo.nodes), len(grafo.edges) ) # es arbol si m = n - 1!

Quiero elegir la raiz que me de el arbol mas bajito... nose porque... porque es mas lindo nomas

In [ ]:
raiz = np.argmin( np.max( [ [ len(k) for k in d.values() ] for d in dict(nx.all_pairs_shortest_path(grafo)).values() ], axis=1 ) )
arbol = nx.bfs_tree( grafo, raiz )

In [ ]:
pos = graphviz_layout(arbol, prog="dot")
pos = { int(k): v for k,v in pos.items()}
nx.draw(arbol, pos, with_labels=True)
plt.show()

In [1]:
from submesh.submesh import SkeletonMesh

skel = SkeletonMesh( 
        '../data/humans/mesh/faust_6890.off', 
        '../data/humans/centerline/faust_6890.txt',
        '../data/humans/centerline/faust_6890.polylines.txt' )


skel.submesh(pointsPerUnit=5)
skel.postprocess()
skel.centerAndNormalize()

skel.saveToJson( 'prueba.json')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


'prueba.json'